# Assignment 1 - Part 2
Student: Duncan Ferguson <br>
Student Id: 871641260 <br>
Class: Comp 4431-1 <br>
Assignment: Assignment 1 part 2 <br>
Date: 10/26/2021 <br>
Group Members from Assignment 5: Emma Bright, Mike Santoro <br>

Association Rules were originally created for mining information for market baskets.  But, they can provide insight into
 other applications also.  In this part of the assignment you will use the mlxtend library functions apriori( )
  and association_rules( ) to explore text data.  Specifically, we will look at the text from three books:

- Sense and Sensibility, by Jane Austin, published 1811
- A Tale of Two Cities, by Charles Dickens, published 1859
- On The Origin of Species, by Charles Darwin, published 1859 (although written over a 20 year period preceding that
date).

All three books were written in english by english people at "roughly" the same time, hence one could assume some
commonality in writing style.

Mining using association rules assumes a database of a large number of transactions with some number of items per
transaction.  I have processed the text of these books to be presented in a similar fashion, where each "transaction"
is now a sentence from the book with the words comma-separated.  Note, some of the sentences in the books are quite
long which would result in intractable run times (and enormous space to hold all possible itemsets). As we saw from
our reading, the number of subsets to be considered increases exponentially with the number of items per transaction.
 For example, assume a sentence has 60 words.   Then we might need to consider all possible subsets of size 60, 59, 58,
  57, ... and 1. The number of these would be:  Choose(60,60) + Choose(60,59) + Choose(60,58) + ...Choose(60,2) +
  Choose(60,1).  Clearly not tractable!

To make this problem tractable I have processed the texts to the they contain at most N words per line (transaction),
 where N = {10,12,14}.  For example, assume N = 14.  I processed the file into a list of sentences.  Then, for each
 sentence, if there were more than 14 words I cut into into at most 14 work chunks.  The beginning of a new sentence
 starts a new line.  I also removed all punctuation and made everything lower case.  As far as text analysis goes,
 this loses some useful information, but it greatly simplifies the problem and this assignment is more about
  understanding how to use association rules and frequent itemsets.

On my computer the files for N=14 ran very quickly, about 5 seconds or less depending on the min-support value used,
 but I have provided the smaller sets just in case you need to use them due to having a slower computer. To figure out
 which set of files to use, I suggest you run your code on the 10 words per line, 12 words per line, and 14 words per
  line data sets in turn to see how fast they run.  If you computer works well on the 14, only use the 14 for all three
   texts.  If you need to use the 10 or 12, only use that value for all three texts.

You are to explore each of the three texts and see what you can determine about the texts individually, and compared to
 each other.  Write up a summary of commonality and differences explaining the parameter values used.  I suggest you
  look at frequent 1-itemsets,  2-itemsets and possibly 3-itemsets  as well as association rules.  As a starting point
  I suggest you start with a min-support value of 0.02 and a min-confidence value of 0.2.  Then increase/decrease both
   as you see fit.  Only consider those association rules that have a lift value > someValue, where someValue is around
    1.1 .. 1.5.

The same values may not work well for each of the datasets, it is fine to use different values for different texts,
just make sure to explain what values you used and whey.

Write up your experiments as a report and upload the report as a .pdf file.  Also attach your code for one of your
experiments (all the others should be quite similar).  You do not need one file that runs everything, how much you
"automate" via python scripting is up to you.

In [88]:
# Adding in all the import statements
from csv import reader
import pandas as pd
from mlxtend.frequent_patterns import apriori, association_rules
from mlxtend.preprocessing import TransactionEncoder

In [89]:
# importing the texts
def fill_book_list(file_name):
    """This function reads in the book"""
    book = []
    with open(file_name, 'r') as read_obj:
        csv_reader = reader(read_obj)
        for row in csv_reader:
            book.append(row)
    return book

book1 = fill_book_list('input_SenseAndSensibility_14.csv')
book2 = fill_book_list('input_TaleOfTwoCities_14.csv')
book3 = fill_book_list('input_OriginOfSpecies_14.csv')

Creating a function for using the transaction encoder to transform the books into an 1-hot boolean encoded numpy array
and convert into a dataframe for the convenience of passing into apriori.

In [90]:
def transact_1_hot(book):
    """Creating transact 1 hot boolean encoded numpy array """
    te = TransactionEncoder()
    te_ary = te.fit(book).transform(book)
    df = pd.DataFrame(te_ary, columns= te.columns_)
    return df

book1_df = transact_1_hot(book1)
book2_df = transact_1_hot(book2)
book3_df = transact_1_hot(book3)

Displaying the head of all three books

In [91]:
book1_df.head()

,200,7000l,_,_a,_all,_almost_,_always_,_am_,_amiably,_and,...,youthere,youthful,youtravel,youwill,youwilloughby,youyet,zeal,zealous,zealously,Ã
0,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [92]:
book2_df.head()

,1757,1767,211792,_,_am_,_are_collected,_at_,_bid_,_do_,_don,...,youso,youth,youthful,youthfulness,youths,youwell,youwill,youworth,youyes,zealous
0,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [93]:
book3_df.head()

,000,1,10,100,1000,101,102,107,107330,107416,...,zealand,zealand100,zealand201,zealand376,zebra,zero,zone,zones,zoological,zoologist
0,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


Creating a function to run the apriori algorithm

In [94]:
def run_apriori(df, min_support_feed=0.02, min_threshold_feed=0.2, lift=1.1):
    """This function runs through the apriori algorithm taking in min support
    min threshold which is confidence and min lift. It returns association rules and frequent items"""
    freq_items = apriori(df, min_support=min_support_feed, use_colnames=True)
    freq_items['length'] = freq_items['itemsets'].apply(lambda x: len(x))
    freq_items.sort_values(["support", "length"], inplace=True, ascending=False)
    rules = association_rules(freq_items, metric="confidence", min_threshold=min_threshold_feed)
    asso_rules = rules[rules["lift"] >= lift].copy()
    asso_rules.sort_values(["lift"], inplace=True, ascending=False)
    return asso_rules, freq_items

book1_rules, book1_freq_items = run_apriori(book1_df)
book2_rules, book2_freq_items = run_apriori(book2_df)
book3_rules, book3_freq_items = run_apriori(book3_df)

Taking a look at frequent 1-itemsets that are both strong and interesting

In [95]:
def filter_freq_items(freq_items,items=1):
    """This function filters the frequent items so that we can look a certain n-itemsets"""
    filtered_freq_items = freq_items[freq_items["length"] == items].copy()
    return filtered_freq_items

The five most common one words for every book with a min-support of 0.02

In [96]:
book1_freq_items_1 = filter_freq_items(book1_freq_items)
book1_freq_items_1.head(5)

,support,itemsets,length
61,0.300081,(to),1
56,0.283155,(the),1
4,0.276672,(and),1
42,0.269650,(of),1
22,0.174845,(her),1


In [97]:
book2_freq_items_1 = filter_freq_items(book2_freq_items)
book2_freq_items_1.head(5)

,support,itemsets,length
44,0.421277,(the),1
3,0.324901,(and),1
34,0.259023,(of),1
50,0.232467,(to),1
0,0.192829,(a),1


In [98]:
book3_freq_items_1 = filter_freq_items(book3_freq_items)
book3_freq_items_1.head(5)

,support,itemsets,length
50,0.522363,(the),1
36,0.434390,(of),1
3,0.283951,(and),1
25,0.257707,(in),1
56,0.225992,(to),1


### Frequent 1-itemsets
When we look at the frequent item sets for 1 word items from the three books we notice a lot of similarities.
All three books have the following words in the top 5;
- The
- To
- And
- Of <br>

Books two and three has a really high support for the word "the". The rankings of the top 5 words varies but there is
nothing that is too surprising here as these words appear in most english sentences. <br>

Now looking into the differences we do find that the support ranking order is different. Listed below are the words that
 appear in the top 5 for one book but not the others:
- "her" (book1)
- "a" (book2)
- "in" (book3)


The five most common two words itemsets for every book with a min-support of 0.02

In [99]:
book1_freq_items_1 = filter_freq_items(book1_freq_items, 2)
book1_freq_items_1.head(5)

,support,itemsets,length
177,0.138651,"(of, the)",2
105,0.098316,"(and, the)",2
190,0.095165,"(to, the)",2
106,0.092284,"(to, and)",2
101,0.089943,"(and, of)",2


In [100]:
book2_freq_items_1 = filter_freq_items(book2_freq_items, 2)
book2_freq_items_1.head(5)

,support,itemsets,length
151,0.171316,"(the, of)",2
91,0.163672,"(the, and)",2
160,0.109771,"(to, the)",2
138,0.105437,"(in, the)",2
88,0.092120,"(and, of)",2


In [101]:
book3_freq_items_1 = filter_freq_items(book3_freq_items, 2)
book3_freq_items_1.head(5)

,support,itemsets,length
162,0.311895,"(of, the)",2
135,0.164190,"(in, the)",2
88,0.163303,"(the, and)",2
181,0.126488,"(to, the)",2
84,0.126118,"(of, and)",2


### Frequent 2-itemsets

Right off the bat we can see some major differences by looking at the support values for the top 5 two itemset words.
 Book3, Darwin, is using a lot of two word itemsets much more. Book1 is showing a lot less support for using the same two
 word itemsets. Jane Austin's novel was probably a bit smoother of a read with more variety. To lay out the similarities
 in black and white below are the similarities and differences for the top 5.

 __Similarities__
 - [Of, The]
 - [The, And]
 - [The, To]
 - [Of, And]

__Differences__
 - [To, And] (Book1)
 - [The, In] (Book2) (Book3)

Once again there are four 2-item sets appear in the top 5 for all three books. Almost close to having all 5 being similiar

The five most common three word itemsets for every book with a min-support of 0.02

In [102]:
book1_freq_items_1 = filter_freq_items(book1_freq_items, 3)
book1_freq_items_1.head(5)

,support,itemsets,length
202,0.046187,"(the, and, of)",3
209,0.043756,"(of, to, the)",3
205,0.030431,"(of, her, the)",3
204,0.029711,"(to, and, the)",3
208,0.029621,"(of, in, the)",3


In [103]:
book2_freq_items_1 = filter_freq_items(book2_freq_items, 3)
book2_freq_items_1.head(5)

,support,itemsets,length
174,0.060757,"(the, and, of)",3
179,0.044129,"(the, in, of)",3
182,0.040032,"(to, the, of)",3
176,0.038455,"(to, the, and)",3
173,0.038377,"(the, in, and)",3


In [104]:
book3_freq_items_1 = filter_freq_items(book3_freq_items, 3)
book3_freq_items_1.head(5)

,support,itemsets,length
193,0.089894,"(of, the, and)",3
205,0.088268,"(of, in, the)",3
215,0.066238,"(of, to, the)",3
192,0.047535,"(the, in, and)",3
190,0.046869,"(of, a, the)",3


### Frequent 3-itemsets

__Similarities__
- [Of, The, And]
- [Of, The, In]
- [Of, The, To]

__Differences__
- [Of, The, Her] (Book1)
- [The, In, And] (Book2, Book3)
- [The, To, And] (Book1, Book2)
- [A, The, Of] (Book3)

After looking at all the Frequent 1,2,3 item sets it becomes apparent that all of these words are the same.
The reason being is that they are all articles, pronouns, prepositions, conjunctions. These words don't really tell
us too much about the text or what is in it. Rather it is just showing us the most common words that are used in almost
every english sentence.

### Looking into the association rules with a min_support=.2 and min_confidence=.02

For right now I'm just going through and print out the association rules so that we can reconfirm that all of these
words are mostly "stop words" that do not add too much understanding to what is inside of the books. These words should
have high lift values and confidence as they are used with a high frequency but do not relay much value.

In [105]:
book1_rules.head()

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
162,(am),(i),0.022508,0.146394,0.020167,0.896000,6.120462,0.016872,8.207748
96,(have),(i),0.069506,0.146394,0.027370,0.393782,2.689878,0.017195,1.408085
72,(you),(i),0.084631,0.146394,0.030701,0.362766,2.478008,0.018312,1.339548
71,(i),(you),0.146394,0.084631,0.030701,0.209717,2.478008,0.018312,1.158280
124,"(a, the)",(of),0.044566,0.269650,0.024579,0.551515,2.045302,0.012562,1.628484


In [106]:
book2_rules.head()

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
132,(mr),(lorry),0.042396,0.025532,0.023089,0.544610,21.330545,0.022007,2.139852
131,(lorry),(mr),0.025532,0.042396,0.023089,0.904321,21.330545,0.022007,10.008511
150,(have),(i),0.055004,0.126241,0.021434,0.389685,3.086829,0.014490,1.431652
140,(is),(it),0.056659,0.130102,0.022301,0.393602,3.025325,0.014930,1.434533
80,(i),(you),0.126241,0.088574,0.031127,0.246567,2.783748,0.019945,1.209698


In [107]:
book3_rules.head()

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
98,(have),(been),0.121535,0.064538,0.033932,0.279197,4.326116,0.026089,1.297806
99,(been),(have),0.064538,0.121535,0.033932,0.525773,4.326116,0.026089,1.852416
142,(is),(it),0.098100,0.074444,0.027870,0.284099,3.816299,0.020567,1.292856
141,(it),(is),0.074444,0.098100,0.027870,0.374379,3.816299,0.020567,1.441608
119,(that),(it),0.147557,0.074444,0.030458,0.206413,2.772737,0.019473,1.166294


The first look at the association rules is filter to have the lift values as the highest. The first book contains
a lot of words [am, have, you] as antecedents to the consequents I. <br>

The second book [mr] followed by [lorry] makes sense as mr is always going to be before lorry, for Mr Lory. <br>

The third book definitely shows a few different rules. With [have] followed by [been] would make sense for
Darwin describing different types of animals and evolution. This can also be shown by less person pronouns being used.

### Filtering rules by support to add an extra look

## Removing the Stop words.
 This way we can go through and figure out other associations That might be more intersting
 Do note that after running through this thing

In [108]:
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS

def remove_stop_words(book):
    """This function goes through and removes the stop words from the list"""
    additional_stop_words = ['s']
    stop_words = ENGLISH_STOP_WORDS.union(additional_stop_words)
    # stop_words = ENGLISH_STOP_WORDS
    new_book = []
    for line in book:
        new_book.append([word for word in line if word not in stop_words])
    return new_book

In [109]:
new_book1 = remove_stop_words(book1)
new_book2 = remove_stop_words(book2)
new_book3 = remove_stop_words(book3)

In [110]:
new_book1_df = transact_1_hot(new_book1)
new_book2_df = transact_1_hot(new_book2)
new_book3_df = transact_1_hot(new_book3)

In [111]:
new_book1_rules, new_book1_freq_items = run_apriori(new_book1_df)
new_book2_rules, new_book2_freq_items = run_apriori(new_book2_df, min_support_feed=.01, min_threshold_feed=.01)
new_book3_rules, new_book3_freq_items = run_apriori(new_book3_df, min_support_feed=.01, min_threshold_feed=.8)

The five most common one words for every book with a min-support of 0.02

In [112]:
new_book1_freq_items_1 = filter_freq_items(new_book1_freq_items)
new_book1_freq_items_1.head(5)

,support,itemsets,length
1,0.057801,(elinor),1
2,0.046457,(marianne),1
4,0.044747,(mrs),1
5,0.032862,(said),1
6,0.021878,(sister),1


In [113]:
new_book2_freq_items_1 = filter_freq_items(new_book2_freq_items)
new_book2_freq_items_1.head(5)

,support,itemsets,length
32,0.047203,(said),1
27,0.042396,(mr),1
22,0.025532,(lorry),1
24,0.021198,(man),1
18,0.019937,(little),1


In [114]:
new_book3_freq_items_1 = filter_freq_items(new_book3_freq_items)
new_book3_freq_items_1.head(5)


,support,itemsets,length
37,0.094773,(species),1
40,0.027722,(varieties),1
15,0.026022,(forms),1
36,0.024691,(selection),1
28,0.023287,(natural),1


In [115]:
new_book1_rules

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
